In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("LogistcRegerssion_Exercise").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/11 14:15:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/11 14:15:24 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


24/03/11 14:15:39 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [2]:
from pyspark.ml.feature import (
    VectorAssembler,
    VectorIndexer,
    OneHotEncoder,
    StringIndexer,
)
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import (
    BinaryClassificationEvaluator,
    MulticlassClassificationEvaluator,
)

In [3]:
df = spark.read.csv("customer_churn.csv", inferSchema=True, header=True)

In [4]:
df.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|      Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|    1|
|     

In [5]:
df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [6]:
df.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [15]:
my_cols = df.select(
    "Age", "Total_Purchase", "Account_Manager", "Years", "Num_Sites", "churn"
)

In [16]:
my_cols.show()

+----+--------------+---------------+-----+---------+-----+
| Age|Total_Purchase|Account_Manager|Years|Num_Sites|churn|
+----+--------------+---------------+-----+---------+-----+
|42.0|       11066.8|              0| 7.22|      8.0|    1|
|41.0|      11916.22|              0|  6.5|     11.0|    1|
|38.0|      12884.75|              0| 6.67|     12.0|    1|
|42.0|       8010.76|              0| 6.71|     10.0|    1|
|37.0|       9191.58|              0| 5.56|      9.0|    1|
|48.0|      10356.02|              0| 5.12|      8.0|    1|
|44.0|      11331.58|              1| 5.23|     11.0|    1|
|32.0|       9885.12|              1| 6.92|      9.0|    1|
|43.0|       14062.6|              1| 5.46|     11.0|    1|
|40.0|       8066.94|              1| 7.11|     11.0|    1|
|30.0|      11575.37|              1| 5.22|      8.0|    1|
|45.0|       8771.02|              1| 6.64|     11.0|    1|
|45.0|       8988.67|              1| 4.84|     11.0|    1|
|40.0|       8283.32|              1|  5

In [17]:
my_final_data = my_cols.dropna()

In [18]:
assembler = VectorAssembler(
    inputCols=[
        "Age",
        "Total_Purchase",
        "Account_Manager",
        "Years",
        "Num_Sites",
    ],
    outputCol="features",
)

In [19]:
output = assembler.transform(my_final_data)

In [13]:
output.show()

+----+--------------+---------------+-----+---------+--------------------+
| Age|Total_Purchase|Account_Manager|Years|Num_Sites|            features|
+----+--------------+---------------+-----+---------+--------------------+
|42.0|       11066.8|              0| 7.22|      8.0|[42.0,11066.8,0.0...|
|41.0|      11916.22|              0|  6.5|     11.0|[41.0,11916.22,0....|
|38.0|      12884.75|              0| 6.67|     12.0|[38.0,12884.75,0....|
|42.0|       8010.76|              0| 6.71|     10.0|[42.0,8010.76,0.0...|
|37.0|       9191.58|              0| 5.56|      9.0|[37.0,9191.58,0.0...|
|48.0|      10356.02|              0| 5.12|      8.0|[48.0,10356.02,0....|
|44.0|      11331.58|              1| 5.23|     11.0|[44.0,11331.58,1....|
|32.0|       9885.12|              1| 6.92|      9.0|[32.0,9885.12,1.0...|
|43.0|       14062.6|              1| 5.46|     11.0|[43.0,14062.6,1.0...|
|40.0|       8066.94|              1| 7.11|     11.0|[40.0,8066.94,1.0...|
|30.0|      11575.37|    

In [20]:
final_data = output.select("features", "churn")

In [21]:
final_data.show()

+--------------------+-----+
|            features|churn|
+--------------------+-----+
|[42.0,11066.8,0.0...|    1|
|[41.0,11916.22,0....|    1|
|[38.0,12884.75,0....|    1|
|[42.0,8010.76,0.0...|    1|
|[37.0,9191.58,0.0...|    1|
|[48.0,10356.02,0....|    1|
|[44.0,11331.58,1....|    1|
|[32.0,9885.12,1.0...|    1|
|[43.0,14062.6,1.0...|    1|
|[40.0,8066.94,1.0...|    1|
|[30.0,11575.37,1....|    1|
|[45.0,8771.02,1.0...|    1|
|[45.0,8988.67,1.0...|    1|
|[40.0,8283.32,1.0...|    1|
|[41.0,6569.87,1.0...|    1|
|[38.0,10494.82,1....|    1|
|[45.0,8213.41,1.0...|    1|
|[43.0,11226.88,0....|    1|
|[53.0,5515.09,0.0...|    1|
|[46.0,8046.4,1.0,...|    1|
+--------------------+-----+
only showing top 20 rows



In [22]:
train_data, test_data = final_data.randomSplit([0.7, 0.3], seed=20)

In [11]:
log_reg_churn = LogisticRegression(featuresCol="features", labelCol="churn")

In [23]:
fit_model = log_reg_churn.fit(train_data)

24/03/11 14:31:03 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/03/11 14:31:03 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


In [32]:
training_sum = fit_model.summary

In [33]:
training_sum.predictions.describe().show()

+-------+------------------+-------------------+
|summary|             churn|         prediction|
+-------+------------------+-------------------+
|  count|               632|                632|
|   mean|0.1661392405063291|0.12341772151898735|
| stddev|0.3725003956577642|0.32917660740301913|
|    min|               0.0|                0.0|
|    max|               1.0|                1.0|
+-------+------------------+-------------------+



In [34]:
pred_and_labels = fit_model.evaluate(test_data)

In [35]:
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[26.0,8787.39,1.0...|    1|[0.54246119410196...|[0.63238476824297...|       0.0|
|[29.0,8688.17,1.0...|    1|[2.45716750347477...|[0.92108401902974...|       0.0|
|[29.0,12711.15,0....|    0|[5.19499697983321...|[0.99448633597886...|       0.0|
|[29.0,13240.01,1....|    0|[6.37613578234584...|[0.99830120486288...|       0.0|
|[30.0,10744.14,1....|    1|[1.58283875446673...|[0.82960617919960...|       0.0|
|[30.0,10960.52,1....|    0|[2.23943977214990...|[0.90373573091218...|       0.0|
|[31.0,5387.75,0.0...|    0|[2.32429478800349...|[0.91086923517753...|       0.0|
|[32.0,5756.12,0.0...|    0|[3.89223992128515...|[0.98000822295783...|       0.0|
|[32.0,6367.22,1.0...|    0|[2.76961518397295...|[0.94101162950612...|       0.0|
|[32.0,8575.71,0

In [27]:
result = fit_model.transform(test_data)

In [28]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="churn")

In [30]:
result.select("prediction", "churn").show()

+----------+-----+
|prediction|churn|
+----------+-----+
|       0.0|    1|
|       0.0|    1|
|       0.0|    0|
|       0.0|    0|
|       0.0|    1|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    1|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
+----------+-----+
only showing top 20 rows



In [31]:
AUC = my_eval.evaluate(result)
AUC

0.7709516691579472

In [36]:
AUC1 = my_eval.evaluate(pred_and_labels.predictions)
AUC1

0.7709516691579472

Predict on new data

In [37]:
final_lr_model = log_reg_churn.fit(final_data)

In [38]:
new_customer = spark.read.csv("new_customers.csv", inferSchema=True, header=True)

In [39]:
new_customer.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [40]:
test_new_customers = assembler.transform(new_customer)

In [41]:
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [42]:
final_result = final_lr_model.transform(test_new_customers)

In [43]:
final_result.show()

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+
|         Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|         Company|            features|       rawPrediction|         probability|prediction|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+
| Andrew Mccall|37.0|       9935.53|              1| 7.71|      8.0|2011-08-29 18:37:54|38612 Johnny Stra...|        King Ltd|[37.0,9935.53,1.0...|[2.22168680572547...|[0.90218015921764...|       0.0|
|Michele Wright|23.0|       7526.94|              1| 9.28|     15.0|2013-07-22 18:19:54|21083 Nicole Junc...|   Cannon-Benson|[23.0,7526.94,1.0...|[-6.2207539991844...|[0.00198380259784...|       

In [45]:
final_result.select("Company", "prediction").show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

